In [1]:
def write_files():
    from random import randint
    with open('d1.txt', 'w') as f1, open('d2.txt', 'w') as f2:
        for _ in range(20):
            rint = randint(1, 100)
            f1.write(f'{rint}\n')
            '''if randint(1,10) > 7:
                rint = randint(1, 100)'''
            f2.write(f'{rint}\n')
        f2.write(f'999\n')
        
write_files()

In [3]:
def compare_files(f1, f2):
    for count, (linef1, linef2) in enumerate(zip(f1, f2)):
        if not linef1 == linef2:
            return count, linef1, linef2
    return None, None, None

In [4]:
file1, file2 = 'd1.txt', 'd2.txt'
with open(file1, 'r') as f1, open(file2, 'r') as f2:
    line_num, f1line, f2line = compare_files(f1, f2)
    if line_num:
        print(f'Mismatch found at line #{line_num}')
        print(f'{file1} had:\n{f1line}')
        print(f'{file2} had:\n{f2line}')

In [52]:
def check_files(*file_handles):
    '''line by line comparison of multiple text files, returns on first detection of
    a difference between any of the files and returns the number of the line it was found
    on and a copy of the line from each file in a list
    
    also detects if any of the files are longer, returns the number of the line where one or
    more files stop, and a copy of the next line of all longer files
    
    if files are identical, returns None and None for the line count and list of lines
    
    arguments are the file handles'''
    
    def next_line(file_handle):
        '''return next line of file or None if no more lines'''
        try:
            line = next(file_handle)
        except StopIteration:  # reached end of file
            return None
        else:
            return line

    count = 0  # file(s) line number 
    while True:
        lines = []  # to store copy of next line of each file
        for file_handle in file_handles:
            lines.append(next_line(file_handle))
        if set(lines) == {None}:  # reached end of all files
            return None, None
        if None in lines:  # reached end of one or more but not all files
            return -count, lines
        if len(set(lines)) > 1:  # line differences found 
            return count, lines
        count += 1

if __name__ == '__main__':
    
    import contextlib

    # need to make sure these files exist
    filenames = 'd1.txt', 'd2.txt', 'd3.txt', 'd4.txt'
    
    with contextlib.ExitStack() as stack:
        try:
            files = [stack.enter_context(open(fname)) for fname in filenames]
        except FileNotFoundError:
            print(f'Could not open all files: {filenames}')
        else:
            count, lines = check_files(*files)
            if count:
                if count > 0:
                    print(f'Difference found at line {count}')
                    for file_handle, line in zip(files, lines):
                        print(f'{file_handle.name} contained:\n{line.rstrip()}')
                elif count < 0:
                    print(f'Different file lenghts found after line {-count-1}')
                    for file_handle, line in zip(files, lines):
                        if line:
                            print(f'{file_handle.name} contained:\n{line.rstrip()}')
                        else:
                            print(f'{file_handle.name} ended')
            else:
                print('No differences found')

Difference found at line 5
d1.txt contained:
43
d2.txt contained:
43
d3.txt contained:
43
d4.txt contained:
46


In [ ]:
length = None
if len(sys.argv) > 1:
    if not sys.argv[1].isnumeric():
        print("You must enter a passphrase length ≥1\n\nTry again!")
    elif int(sys.argv[1]) < 1:
        print("You must enter a passphrase length ≥1\n\nTry again!")
    else:
        passphrase = ' '.join(generatePassphrase(sys.argv[1]));
        sendToClipboard(passphrase)
else:
    passphrase = ' '.join(generatePassphrase());
    sendToClipboard(passphrase)